Watch DM progress

In [1]:
import os
import time
import numpy as np
import xarray as xr
result_dir = '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/'
path = os.path.join(result_dir, 'samples')
in_ds = xr.load_dataset(path + '/uq_modal_beam.nc')
out_ds = xr.load_dataset(path+'uq_modal_beam_out.nc')

In [1]:
np.isnan(out_ds._runtimes.data)[0][0]

In [33]:
for i in range(3938021,7599653): # 3938021 is the first invalid # 3625888 is broken, is not broken?
    if not i%250:
        print('.', end='', flush=True)
    jid = in_ds.ids[i].item()
    res_dir = os.path.join(path,jid,f'{jid}_mechanical.npz')
    arr = np.load(res_dir, allow_pickle=True)
    assert arr['self.struct_parms'].item()['dy_tmd'] == in_ds['dD'][i].item()

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

AssertionError: 

In [34]:
display(i)

3938021

In [38]:
jid = in_ds.ids[i].item()
res_dir = os.path.join(path,jid,f'{jid}_mechanical.npz')
print(time.ctime(os.stat(res_dir).st_mtime))
print(time.ctime(os.stat(res_dir).st_ctime))

Mon Nov  7 09:21:49 2022
Mon Nov  7 09:21:49 2022


- 9,999,693 = 729 * 13,717 # total samples
- 3,930,727 / 729 = 5,391.9 # last datamanager sample with runtime not nan -> start_ale=5390 is the last good sample when plotting in estimate_imp
- 6,581,412 = 729 * 9,028 # last datamanager sample with results not nan
- 3,938,021 / 729 = 5,401.9 # last sample, that progress watcher marks valid, probably the last 7294 samples were not saved into db -> it is bad when plotting in estimate_imp

In [68]:
for ret_name in ['damp_freqs','zetas','frf']:
    
    if ret_name == 'frf':
        # continue
        inds = range(1025*2)
    else:
        # continue
        inds = range(14)
    all_times = []
    all_dirs = []
    all_progr = []
    for ind in inds:
        if ret_name == 'frf':
            ret_ind = {'frequencies':ind//2, 'space':ind%2}
        else:
            ret_ind = {'modes':ind}
            
        ret_dir = f'{ret_name}-{".".join(str(e) for e in ret_ind.values())}'
        fname = os.path.join(result_dir, 'estimations', f'{ret_dir}/polyuq_imp.npz')
        # print(ret_dir)
        if os.path.exists(fname):
            all_times.append(os.stat(fname).st_mtime)
            all_dirs.append(ret_dir)
            intp_err = np.load(fname)['self.intp_errors']
            nan_ind = np.where(np.isnan(intp_err))[0]
            if len(nan_ind):
                all_progr.append(np.min(nan_ind))
            else:
                all_progr.append(len(intp_err))
    
    
    for ret_time,ret_dir, progr in sorted(zip(all_times, all_dirs, all_progr)):
        print(f'{ret_dir} @ {progr} on {time.ctime(ret_time)}')

damp_freqs-6 @ 5390 on Mon Feb  6 20:03:13 2023
damp_freqs-8 @ 5390 on Mon Feb  6 20:07:40 2023
damp_freqs-12 @ 5390 on Mon Feb  6 20:11:20 2023
damp_freqs-1 @ 5390 on Mon Feb  6 20:13:24 2023
damp_freqs-0 @ 5390 on Mon Feb  6 20:16:07 2023
damp_freqs-10 @ 5390 on Mon Feb  6 20:17:12 2023
damp_freqs-11 @ 5390 on Mon Feb  6 20:24:43 2023
damp_freqs-2 @ 5390 on Mon Feb  6 20:29:20 2023
damp_freqs-9 @ 5390 on Mon Feb  6 20:33:11 2023
damp_freqs-4 @ 5390 on Mon Feb  6 20:34:15 2023
damp_freqs-7 @ 5390 on Mon Feb  6 20:44:08 2023
damp_freqs-5 @ 5390 on Mon Feb  6 20:57:54 2023
damp_freqs-3 @ 5390 on Mon Feb  6 21:07:07 2023
damp_freqs-13 @ 5390 on Tue Feb  7 05:29:10 2023
zetas-4 @ 5390 on Mon Feb  6 20:47:46 2023
zetas-0 @ 5390 on Mon Feb  6 20:56:28 2023
zetas-7 @ 5390 on Mon Feb  6 21:07:23 2023
zetas-5 @ 5390 on Mon Feb  6 21:09:43 2023
zetas-1 @ 5390 on Mon Feb  6 21:13:26 2023
zetas-2 @ 5390 on Mon Feb  6 21:15:26 2023
zetas-6 @ 5390 on Mon Feb  6 21:22:01 2023
zetas-3 @ 5390 on Mon F

In [67]:
ret_dir = 'frf-585.0'
fname = os.path.join(result_dir, 'estimations', f'{ret_dir}/polyuq_imp.npz')
os.stat(fname).st_mtime > 1675778682.0

True